In [43]:
import os

In [44]:
%pwd

'c:\\Users\\madar'

In [45]:
os.chdir("../")

In [46]:
%pwd

'c:\\Users'

In [55]:
# entity means written type of the function which is done using dataclass
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file:Path
    unzip_dir:Path


In [66]:
import sys
import os
from pathlib import Path

# Use absolute path to project root
project_root = Path(r"C:\Users\madar\Text-Summarizer")
print(f"Project root: {project_root}")

# Change to project root
os.chdir(project_root)
print(f"Current directory: {Path.cwd()}")

# Verify config files exist
config_file = project_root / "config" / "config.yaml"
params_file = project_root / "params.yaml"

print(f"Config file exists: {config_file.exists()}")
print(f"Params file exists: {params_file.exists()}")

# Add src to path
sys.path.insert(0, str(project_root / "src"))

# Define paths
CONFIG_FILE_PATH = config_file
PARAMS_FILE_PATH = params_file

from textSummarizer.utils.common import read_yaml, create_directories

Project root: C:\Users\madar\Text-Summarizer
Current directory: C:\Users\madar\Text-Summarizer
Config file exists: True
Params file exists: True


In [67]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [68]:
import os
from pathlib import Path
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [69]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [70]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-20 13:28:53,242: INFO: common: yaml file: C:\Users\madar\Text-Summarizer\config\config.yaml loaded successfully]
[2025-12-20 13:28:53,246: INFO: common: yaml file: C:\Users\madar\Text-Summarizer\params.yaml loaded successfully]
[2025-12-20 13:28:53,249: INFO: common: created directory at: artifacts]
[2025-12-20 13:28:53,252: INFO: common: created directory at: artifacts/data_ingestion]
[2025-12-20 13:28:59,448: INFO: 12640755: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 40CF:23F100:1D44D1:257075:6946573E
Accept-Ranges: bytes
Date: Sat, 20 Dec 2025 07:58:55 GMT